In [1]:
import sys
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/Downloads/jax/build
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision as tv

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

num_workers = os.cpu_count()
print(num_workers)

cuda:0
6


In [4]:
# parameters for trained CNN
root = os.path.join('.', 'dataset_root')
# mean, std = [0.13066046], [0.30150425] # based on training set
batch_size = 64
lr = 0.001
momentum=0.9
epochs = 30

In [5]:
def imshow(tensor_grid, mean=0., std=1., title=None):
    assert isinstance(tensor_grid, torch.Tensor)
    assert len(tensor_grid.size()) == 4, \
        f'For a batch of images only, {tensor_grid.size()} '
    
    tensor_grid = tv.utils.make_grid(tensor_grid)
    grid = tensor_grid.numpy().transpose((1,2,0))
    grid = std * grid + mean
    grid = np.clip(grid, 0, 1)
    plt.imshow(grid)
    
    if title is not None:
        plt.title(title)
        
    plt.pause(0.001)

In [6]:
# prepare data
# foolbox model expects raw numpy array as image
transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
#         tv.transforms.Normalize(mean, std)
])
train_dataset = tv.datasets.MNIST(
    root,
    train=True, 
    download=True,
    transform=transform)
test_dataset = tv.datasets.MNIST(
    root,
    train=False, 
    download=True,
    transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(int(((28 - (3-1)) / 2)**2 * 8), 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [8]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0.
    corrects = 0
    
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        batch_size = x.size(0)
        
        optimizer.zero_grad()
        output = model(x)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()

        # for display
        total_loss += loss.item() * batch_size
        preds = output.max(1, keepdim=True)[1]
        corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(train_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [9]:
def validate(model, loader):
    model.eval()
    total_loss = 0.
    corrects = 0
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            output = model(x)
            loss = F.nll_loss(output, y)
            total_loss += loss.item() * batch_size
            preds = output.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(test_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [10]:
# NOTE: NO GPU AT SCHOOL!
model = Net()
model_softmax = torch.nn.Sequential(
    model,
    torch.nn.LogSoftmax(dim=1)
)
model_softmax.to(device)
optimizer = torch.optim.SGD(model_softmax.parameters(), lr=lr, momentum=momentum)

In [11]:
since = time.time()
for epoch in range(epochs):
    start = time.time()
    tr_loss, tr_acc = train(model_softmax, train_loader, optimizer)
    va_loss, va_acc = validate(model_softmax, test_loader)
    
    time_elapsed = time.time() - start
    print(('[{:2d}] {:.0f}m {:.1f}s Train Loss: {:.4f} Accuracy: {:.4f}%, ' +
        'Test Loss: {:.4f} Accuracy: {:.4f}%').format(
            epoch+1, time_elapsed // 60, time_elapsed % 60,
            tr_loss, tr_acc*100.,
            va_loss, va_acc*100.))
    
time_elapsed = time.time() - since
print('Total run time: {:.0f}m {:.1f}s'.format(
    time_elapsed // 60,
    time_elapsed % 60))

[ 1] 0m 2.1s Train Loss: 1.1427 Accuracy: 70.4567%, Test Loss: 0.3623 Accuracy: 89.5400%
[ 2] 0m 2.1s Train Loss: 0.3948 Accuracy: 89.5367%, Test Loss: 0.2905 Accuracy: 91.5700%
[ 3] 0m 2.1s Train Loss: 0.3293 Accuracy: 91.1733%, Test Loss: 0.2464 Accuracy: 92.7900%
[ 4] 0m 2.1s Train Loss: 0.2825 Accuracy: 92.3900%, Test Loss: 0.2305 Accuracy: 93.3800%
[ 5] 0m 2.1s Train Loss: 0.2475 Accuracy: 93.3000%, Test Loss: 0.1913 Accuracy: 94.4200%
[ 6] 0m 2.2s Train Loss: 0.2201 Accuracy: 93.9767%, Test Loss: 0.1789 Accuracy: 94.6400%
[ 7] 0m 2.2s Train Loss: 0.1974 Accuracy: 94.5400%, Test Loss: 0.1629 Accuracy: 94.8800%
[ 8] 0m 2.2s Train Loss: 0.1799 Accuracy: 95.0650%, Test Loss: 0.1494 Accuracy: 95.6500%
[ 9] 0m 2.1s Train Loss: 0.1641 Accuracy: 95.4783%, Test Loss: 0.1395 Accuracy: 95.8100%
[10] 0m 2.1s Train Loss: 0.1524 Accuracy: 95.8167%, Test Loss: 0.1293 Accuracy: 96.0900%
[11] 0m 2.1s Train Loss: 0.1421 Accuracy: 96.0633%, Test Loss: 0.1188 Accuracy: 96.2700%
[12] 0m 2.1s Train Lo

In [12]:
torch.save(model.state_dict(), 'mnist_simple_nosoftmax.pt')
torch.save(model_softmax.state_dict(), 'mnist_simple_full.pt')